# Najboljši filmi

Vzorčni projekt pri Programiranju 1 z namenom analize ocen, žanrov in igralcev izbranih filmov.

## Predpriprava

Na začetku naložimo paket in razpredelnice. Če CSV datoteke še ne obstajajo, jih najprej ustvarimo s pomočjo skripte `ustvari_csv.py`.

In [8]:
import pandas as pd

pd.options.display.max_rows = 30
pd.options.display.max_columns = 16

filmi = pd.read_csv('csv-datoteke/filmi.csv', index_col='id')
igralci = pd.read_csv('csv-datoteke/igralci.csv', index_col='id')
zanri = pd.read_csv('csv-datoteke/zanri.csv', index_col='id')
vloge = pd.read_csv('csv-datoteke/vloge.csv')
dolocitve_zanra = pd.read_csv('csv-datoteke/dolocitve_zanra.csv')
rotten = pd.read_csv('csv-datoteke/rotten.csv', index_col='id')

In [ ]:
%matplotlib

Večina podatkov je zajetih s strani IMDB in že ustrezno očiščenih. Dodati moramo le še ocene kritikov, ki smo jih zajeli s strani Rotten Tomatoes. Edini skupni atribut, po katerem lahko ujemamo podatke, sta naslov in leto filma, pri čemer moramo upoštevati, da se naslovi lahko razlikujejo (ločila, naglasi, …). Zato pri obeh razpredelnicah dodamo še stolpec z očiščenim imenom.

In [6]:
# knjižnico unidecode moramo poprej namestiti
from unidecode import unidecode

def pocisti_naslov(naslov):
    # uredimo Unicode (npr. š se pretvori v s + strehica)
    naslov = unidecode(naslov)
    # naslov pretvorimo v ASCII, s čimer se znebimo naglasov
    naslov = naslov.lower()
    # če je na začetku "the", "a" ali "an", jih pobrišemo
    if naslov[:2] == 'a ':
        naslov = naslov[2:]
    elif naslov[:3] == 'an ':
        naslov = naslov[3:]
    elif naslov[:4] == 'the ':
        naslov = naslov[4:]
    # pobrišemo morebitni alternativni naslov (kar sledi prvemu oklepaju)
    if '(' in naslov:
        naslov = naslov[:naslov.find('(')]
    # pobrišemo vse znake, ki niso črke
    naslov = ''.join(znak for znak in naslov if 'a' <= znak <= 'z')
    return naslov

rotten['pociscen_naslov'] = rotten['naslov'].map(pocisti_naslov)
filmi['pociscen_naslov'] = filmi['naslov'].map(pocisti_naslov)

## Analiza

### Korelacija med ocenami

Ali so ocene gledalcev in kritikov usklajene med sabo?

In [4]:
pd.merge(filmi, rotten, on=['pociscen_naslov', 'leto'], how='left', suffixes=('','_rotten')).plot.scatter('ocena', 'ocena_rotten')

### Povprečne ocene igralcev

In [7]:
vloge\
    .join(filmi, on='film')\
    .groupby('igralec')\
    .mean()\
    .sort_values('ocena')\
    .join(igralci)

,film,leto,ocena,ime
igralec,,,,
86268,40457,1948.0,7.00,Janet Blair
2013,40546,1948.0,7.00,Charles Coburn
85782,49006,1956.0,7.00,Sidney Blackmer
526485,43660,1951.0,7.00,William Lundigan
898687,38360,1946.0,7.00,June Vincent
583093,47417,1954.0,7.00,Emile Meyer
141681,40457,1948.0,7.00,Janis Carter
137023,23818,1933.0,7.00,Philip Carey
...,...,...,...,...


### Popularnost žanrov

In [ ]:
filmi['desetletje'] = 10 * (filmi['leto'] // 10)
dolocitve_zanra\
    .join(filmi, on='film')\
    .groupby(['desetletje', 'zanr'])\
    .size()\
    .unstack()\
    .apply(lambda x: x / x.sum(), axis=1)\
    .plot(kind='area')

## Avtomatično določanje žanrov

In [21]:
import orodja


zanri_korenov = pd.read_csv('csv-datoteke/zanri_korenov.csv', index_col='koren') + 1
verjetnost_zanra = dolocitve_zanra.groupby('zanr').size() / len(filmi)
verjetnost_korena_pri_zanru = zanri_korenov / zanri_korenov.sum()

def doloci_zanre(opis):
    koreni_opisa = sorted(orodja.koreni_besed(opis))
    verjetnosti = verjetnost_korena_pri_zanru[verjetnost_korena_pri_zanru.index.isin(koreni_opisa)]
    faktorji_zanrov = verjetnosti.prod() * verjetnost_zanra
    faktorji_zanrov /= max(faktorji_zanrov)
    return faktorji_zanrov.sort_values(ascending=False).head(5)

In [22]:
doloci_zanre("A group of intergalactic criminals are forced to work together to stop a fanatical warrior from taking control of the universe.")

action       1.000000
adventure    0.211980
drama        0.021111
thriller     0.007091
sci_fi       0.007005
dtype: float64

In [23]:
doloci_zanre("Earth's mightiest heroes must come together and learn to fight as a team if they are to stop the mischievous Loki and his alien army from enslaving humanity.")

action       1.000000
sci_fi       0.110356
thriller     0.023182
adventure    0.007143
comedy       0.006247
dtype: float64

In [28]:
doloci_zanre("When Dr Jane Foster gets cursed with a powerful object, Thor must protect it before an army and its ruthless leader try to get their hands on it to take over the remains of Earth.")

drama        1.000000
adventure    0.050116
comedy       0.035726
thriller     0.010064
action       0.004502
dtype: float64